In [1]:
import numpy as np
import time
try:
    import gymnasium as gym
except ImportError:
    import gym

In [2]:
custom_map = [
    'SFFHFF',
    'HFFFFF',
    'FFHFFF',
    'HHFHHF',
    'FFFGFF'
]
map_desc = custom_map

In [3]:
# Create env with human rendering so a window appears when stepping
env = gym.make(
    'FrozenLake-v1',
    is_slippery=True,# True for stochastic environment
    desc=map_desc,
    render_mode='human'
)

In [4]:
# Access MDP transitions and spaces
P = env.unwrapped.P  # dict: P[s][a] -> list of (prob, next_state, reward, terminated)
nS = env.observation_space.n
nA = env.action_space.n

# Hyperparameters
gamma = 0.99
theta = 1e-8


print(f"Number of states: {nS}, Number of actions: {nA}")

Number of states: 30, Number of actions: 4


In [5]:
# Policy evaluation: given a policy pi (array of action indices), compute V

def policy_evaluation(pi, V=None, gamma: float = gamma, theta: float = theta):
    if V is None:
        V = np.zeros(nS, dtype=np.float64)
    else:
        V = np.array(V, dtype=np.float64, copy=True)

    while True:
        delta = 0.0
        for s in range(nS):
            v_old = V[s]
            a = pi[s]
            v_new = 0.0
            for (prob, ns, r, done) in P[s][a]:
                v_new += prob * (r + gamma * (0.0 if done else V[ns]))
            V[s] = v_new
            delta = max(delta, abs(v_old - v_new))
        if delta < theta:
            break
    return V


In [6]:
# Policy improvement: greedy w.r.t. V

def policy_improvement(V, gamma: float = gamma):
    pi = np.zeros(nS, dtype=int)
    for s in range(nS):
        q = np.zeros(nA, dtype=np.float64)
        for a in range(nA):
            for (prob, ns, r, done) in P[s][a]:
                q[a] += prob * (r + gamma * (0.0 if done else V[ns]))
        pi[s] = int(np.argmax(q))
    return pi


In [7]:
# Full policy iteration loop

def policy_iteration(gamma: float = gamma, theta: float = theta):
    # Step 1: Initialize policy randomly and value function
    pi = np.random.randint(0, nA, size=nS, dtype=int)
    V = np.zeros(nS, dtype=np.float64)

    iteration = 0
    while True:
        iteration += 1
        # Step 2: Evaluate current policy
        V = policy_evaluation(pi, V, gamma=gamma, theta=theta)
        # Step 3: Improve the policy using the current value function
        new_pi = policy_improvement(V, gamma=gamma)
        # Step 4: Check if policy has changed
        policy_stable = np.array_equal(pi, new_pi)
        # Step 5: Update policy
        pi = new_pi
        # Step 6: Stop if stable (converged)
        if policy_stable:
            break

    return pi, V, iteration


In [8]:
# One episode run to see the window

def run_episode(env, pi):
    obs, info = env.reset()
    terminated = False
    truncated = False
    total_reward = 0.0
    steps = 0
    while not (terminated or truncated):
        a = int(pi[obs])
        obs, r, terminated, truncated, info = env.step(a)
        total_reward += r
        steps += 1
    return total_reward, steps, terminated, truncated


In [9]:
# Execute and display results
pi_opt, V_opt, iters = policy_iteration(gamma=gamma, theta=theta)

In [10]:
# Display optimal values and policy, then run one episode with rendering
side = int(np.sqrt(nS))
print(f"Converged in {iters} erations")

# Derive shape
rows_map = len(map_desc)
cols_map = len(map_desc[0]) if rows_map > 0 else 0

print('\nOptimal V (grid):')
if rows_map * cols_map == nS and rows_map > 0:
    V_grid = V_opt.reshape(rows_map, cols_map)
    for r in range(rows_map):
        print(' '.join(f'{V_grid[r, c]:>6.3f}' for c in range(cols_map)))
else:
    per_row = max(6, int(np.sqrt(nS)))
    for i in range(0, nS, per_row):
        print(' '.join(f'{v:>6.3f}' for v in V_opt[i:i+per_row]))

Converged in 9 erations

Optimal V (grid):
 0.254  0.262  0.277  0.000  0.665  0.702
 0.000  0.213  0.301  0.422  0.648  0.724
 0.041  0.084  0.000  0.329  0.576  0.767
 0.000  0.000  0.307  0.000  0.000  0.833
 0.851  0.877  0.930  0.000  0.953  0.925


In [11]:
arrow_map = {0: '←', 1: '↓', 2: '→', 3: '↑'}

print("Optimal Policy:")
arrows = [arrow_map[a] for a in pi_opt]
if rows_map * cols_map == nS and rows_map > 0:
    grid = np.array(arrows).reshape(rows_map, cols_map)
    for r in range(rows_map):
        print(' '.join(grid[r]))
else:
    print(' '.join(arrows))


Optimal Policy:
↑ ↑ ← ← → →
← → ↑ ↓ → →
↓ ← ← → ↑ →
← ← ← ← ← →
↓ ↓ ↓ ← ↓ ↓


In [12]:
tr, steps, term, trunc = run_episode(env, pi_opt)
print(f'\nEpisode -> reward: {tr}, steps: {steps}, terminated: {term}, truncated: {trunc}')

# Keep window visible briefly and pump events to avoid 'Not Responding'
try:
    import pygame
    for _ in range(100):  # ~3 seconds
        pygame.event.pump()
        time.sleep(0.03)
except Exception:
    time.sleep(3.0)

env.close()

C:\Users\LENOVO i7\AppData\Local\Programs\Python\Python312\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists



Episode -> reward: 1.0, steps: 73, terminated: True, truncated: False
